In [1]:
#Entrega 1 Miguel Chaveinte

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import mode

In [3]:
#Leemos el dataset
data = pd.read_csv("C:/Users/migue/OneDrive/Escritorio/MINERIA/python-teodoro/practica1/seeds_dataset.txt", sep="\t")

In [4]:
data.head()

,A,P,C,Len,Wid,Asy,Len2,Wheat
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [5]:
data.columns

Index(['A', 'P', 'C', 'Len', 'Wid', 'Asy', 'Len2', 'Wheat'], dtype='object')

In [6]:
#Separamos variable objetivo y estandarizamos datos
y=data.Wheat
y=np.array(y)
X = StandardScaler().fit_transform(data.iloc[:,:-1])
print(np.unique(y))
print('-----')
print(X)

[1 2 3]
-----
[[ 1.42097769e-01  2.15462437e-01  6.06017918e-05 ...  1.41701823e-01
  -9.86151745e-01 -3.83577423e-01]
 [ 1.11880257e-02  8.22375713e-03  4.28515270e-01 ...  1.97432229e-01
  -1.78816620e+00 -9.22013487e-01]
 [-1.92066576e-01 -3.60200562e-01  1.44238325e+00 ...  2.08047544e-01
  -6.67479334e-01 -1.18919199e+00]
 ...
 [-5.67570840e-01 -6.90247348e-01  7.33948301e-01 ... -7.06044846e-02
   3.07658816e+00 -7.18060432e-01]
 [-1.03608992e+00 -1.03564515e+00 -8.01701104e-01 ... -1.12152071e+00
  -6.81351965e-02 -7.42534799e-01]
 [-8.77620233e-01 -9.35863561e-01 -1.10234659e-01 ... -7.55292327e-01
   1.29122264e+00 -7.03783718e-01]]


In [7]:
# Creación de los folds

K=10
cv_strat = StratifiedKFold(n_splits=K,shuffle=True)

max_clusters= 20
min_clusters= np.unique(y).shape[0]  #el minimo es el numero de posibilidades de la clase
k = list(range(min_clusters,max_clusters+1))
k

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [16]:
tabla = pd.DataFrame(columns=k, data=np.zeros((K,max_clusters-min_clusters+1)),dtype=float)  #ccreamos tabla de resultados
tabla.columns.name='Clusters'
tabla.index.name='CV'

for c in k:
    kmedias = KMeans(n_clusters=int(c), random_state=0) #algoritmo k means por cada num de clusters
    cv=0
    for train_index, test_index in cv_strat.split(X,y):
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        kmedias.fit(X_train) #entrenamos con la carpeta correspondiente
        moda=-1 * np.ones(int(c),dtype='int') #moda por cada cluster
        
        for i in range(int(c)):
             moda[i] = mode(y_train[np.argwhere(kmedias.labels_== i)],keepdims=False)[0].astype(int) #vemos clase mayoritaria en cada cluster
        y_test_predict = moda[kmedias.predict(X_test)]
        print(kmedias.predict(X_test))
        print('......')
        print(y_test_predict)
        print('-------')
        tabla.loc[cv,c] = accuracy_score(y_test, y_test_predict)
        cv +=1

tabla

[2 2 2 2 2 2 2 1 1 1 1 1 1 1 0 0 0 0 0 0 2]
......
[1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 1]
-------
[2 2 2 2 2 0 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
......
[1 1 1 1 1 3 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]
-------
[2 1 2 2 2 2 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
......
[1 2 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]
-------
[0 0 0 0 0 1 0 1 1 1 1 0 1 1 2 2 2 2 2 2 2]
......
[1 1 1 1 1 2 1 2 2 2 2 1 2 2 3 3 3 3 3 3 3]
-------
[2 2 2 2 2 2 2 1 1 1 1 1 1 1 0 0 0 2 0 0 2]
......
[1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 1 3 3 1]
-------
[0 1 0 0 0 0 2 1 1 1 1 1 1 0 2 2 2 2 2 2 2]
......
[1 2 1 1 1 1 3 2 2 2 2 2 2 1 3 3 3 3 3 3 3]
-------
[0 2 2 2 2 2 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
......
[3 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3]
-------
[2 2 2 2 2 0 2 1 1 1 1 1 1 1 0 0 0 0 0 2 0]
......
[1 1 1 1 1 3 1 2 2 2 2 2 2 2 3 3 3 3 3 1 3]
-------
[2 2 2 2 2 0 2 1 1 1 1 1 1 1 0 0 0 0 0 2 0]
......
[1 1 1 1 1 3 1 2 2 2 2 2 2 2 3 3 3 3 3 1 3]
-------
[0 0 0 0 0 0 0 1 1 1 1 0 0 0 2 2 2 2 2 2 2]
......
[1 1 1 1 1 1 1 2 2 2 2

Clusters,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
CV,,,,,,,,,,,,,,,,,,
0,0.952381,0.952381,0.952381,0.714286,0.857143,0.904762,0.809524,0.904762,0.714286,0.857143,0.809524,0.904762,0.952381,0.904762,0.857143,0.857143,0.904762,0.952381
1,0.952381,0.904762,0.809524,0.904762,0.809524,0.857143,0.857143,0.952381,0.904762,0.857143,0.952381,0.952381,0.857143,0.952381,0.952381,0.952381,0.857143,0.952381
2,0.952381,0.809524,0.761905,0.904762,0.904762,0.952381,1.000000,0.857143,0.952381,0.904762,0.904762,1.000000,0.904762,0.952381,0.904762,0.857143,1.000000,0.857143
3,0.904762,0.809524,0.809524,0.809524,0.857143,0.952381,0.857143,0.904762,0.952381,0.857143,0.857143,0.952381,0.904762,0.809524,0.952381,0.809524,1.000000,0.952381
4,0.904762,0.952381,0.904762,0.904762,0.857143,0.952381,0.857143,0.809524,0.952381,0.904762,0.857143,0.904762,0.952381,0.809524,0.904762,0.857143,0.809524,0.904762
5,0.857143,0.952381,0.904762,0.904762,0.952381,0.952381,1.000000,0.904762,0.952381,0.952381,0.952381,0.809524,0.857143,0.952381,0.904762,0.857143,0.809524,0.857143
6,0.952381,0.904762,0.809524,1.000000,0.857143,0.761905,0.857143,0.952381,0.952381,0.904762,0.857143,0.857143,0.952381,1.000000,0.857143,1.000000,0.952381,0.904762
7,0.904762,0.952381,0.952381,0.904762,0.952381,0.857143,0.952381,0.857143,0.761905,0.952381,0.952381,0.857143,0.904762,0.952381,0.952381,0.904762,0.904762,0.952381
8,0.904762,0.952381,0.952381,0.809524,0.857143,0.857143,0.904762,0.857143,0.857143,0.952381,0.952381,1.000000,0.952381,0.904762,0.904762,0.952381,0.809524,0.904762


In [11]:
tabla.mean()

Clusters
3     0.914286
4     0.904762
5     0.880952
6     0.866667
7     0.885714
8     0.890476
9     0.880952
10    0.885714
11    0.895238
12    0.880952
13    0.885714
14    0.914286
15    0.885714
16    0.904762
17    0.885714
18    0.909524
19    0.895238
20    0.904762
dtype: float64

In [12]:
print("El k optimo es: ", tabla.columns[tabla.mean(axis=0).argmax()], " con una tasa de aciertos: ", tabla.mean(axis=0).max())

El k optimo es:  3  con una tasa de aciertos:  0.9142857142857143
